In [1]:
#import kagglehub
import yaml
import clickhouse_connect
import pandas as pd
import os

## Load Variables from Docker-Compose YAML

In [2]:
with open("docker-compose.yml", 'r') as ymlfile:
    cfg = yaml.safe_load(ymlfile)

ch_user = cfg['services']['clickhouse']['environment']['CLICKHOUSE_USER']
ch_password = cfg['services']['clickhouse']['environment']['CLICKHOUSE_PASSWORD']

In [3]:
client = clickhouse_connect.get_client(host='localhost', username=ch_user, password=ch_password)

In [4]:
client.command("CREATE DATABASE IF NOT EXISTS datasets")

## World Energy consumption

Although the dataset starts in 1900, much of the data we are interested in was not collected until decades later. For certain data we want to show about Spain, we will use data from 1980 onwards.


In [ ]:
# df = pd.read_csv("datasets/world_energy_consumption.csv")

In [ ]:
# df = df[df["year"] > 1980]

In [ ]:
# df['year'] = pd.to_datetime(df['year'], format='%Y')

In [5]:
folder = "sql/"
for sql_file in os.listdir(folder):
    print(f"Executing {sql_file}...")
    with open(folder + sql_file) as f:
        string_file = f.read()
    client.command(string_file)

Executing iso.sql...
Executing world_energy_consumption.sql...


In [6]:
# Recreate the DataFrame from scratch
df = pd.read_csv("datasets/world_energy_consumption.csv")
df = df[df["year"] > 1980]
df['year'] = pd.to_datetime(df['year'], format='%Y')

# Verify DataFrame structure
print("Columns:", df.columns.tolist())
print("Shape:", df.shape)



Columns: ['country', 'year', 'iso_code', 'population', 'gdp', 'biofuel_cons_change_pct', 'biofuel_cons_change_twh', 'biofuel_cons_per_capita', 'biofuel_consumption', 'biofuel_elec_per_capita', 'biofuel_electricity', 'biofuel_share_elec', 'biofuel_share_energy', 'carbon_intensity_elec', 'coal_cons_change_pct', 'coal_cons_change_twh', 'coal_cons_per_capita', 'coal_consumption', 'coal_elec_per_capita', 'coal_electricity', 'coal_prod_change_pct', 'coal_prod_change_twh', 'coal_prod_per_capita', 'coal_production', 'coal_share_elec', 'coal_share_energy', 'electricity_demand', 'electricity_generation', 'electricity_share_energy', 'energy_cons_change_pct', 'energy_cons_change_twh', 'energy_per_capita', 'energy_per_gdp', 'fossil_cons_change_pct', 'fossil_cons_change_twh', 'fossil_elec_per_capita', 'fossil_electricity', 'fossil_energy_per_capita', 'fossil_fuel_consumption', 'fossil_share_elec', 'fossil_share_energy', 'gas_cons_change_pct', 'gas_cons_change_twh', 'gas_consumption', 'gas_elec_per_c

In [7]:
df['population'] = df['population'].astype('UInt64')
df['iso_code'] = df['iso_code'].astype('str')
df['country'] = df['country'].astype('str')

In [ ]:
# for col in df.columns:
#     print(f"{col}: {df[col].dtype}")

country: object
year: datetime64[ns]
iso_code: object
population: UInt64
gdp: float64
biofuel_cons_change_pct: float64
biofuel_cons_change_twh: float64
biofuel_cons_per_capita: float64
biofuel_consumption: float64
biofuel_elec_per_capita: float64
biofuel_electricity: float64
biofuel_share_elec: float64
biofuel_share_energy: float64
carbon_intensity_elec: float64
coal_cons_change_pct: float64
coal_cons_change_twh: float64
coal_cons_per_capita: float64
coal_consumption: float64
coal_elec_per_capita: float64
coal_electricity: float64
coal_prod_change_pct: float64
coal_prod_change_twh: float64
coal_prod_per_capita: float64
coal_production: float64
coal_share_elec: float64
coal_share_energy: float64
electricity_demand: float64
electricity_generation: float64
electricity_share_energy: float64
energy_cons_change_pct: float64
energy_cons_change_twh: float64
energy_per_capita: float64
energy_per_gdp: float64
fossil_cons_change_pct: float64
fossil_cons_change_twh: float64
fossil_elec_per_capita:

In [8]:
client.insert_df('datasets.world_energy_consumption', df)

DatabaseError: Received ClickHouse exception, code: 1001, server response: std::exception. Code: 1001, type: std::__1::filesystem::filesystem_error, e.what() = filesystem error: in rename: Permission denied ["/var/lib/clickhouse/store/af8/af873437-3883-4bb4-b6f2-dd13322b0246/tmp_insert_all_1_1_0/"] ["/var/lib/clickhouse/store/af8/af873437-3883-4bb4-b6f2-dd13322b0246/all_1_1_0/"]
Cannot print extra info for Poco::Exception (version 25.8.10.7 (official build)) (for url http://localhost:8123)

Insertar equivalente country codes

In [26]:
df_iso = pd.read_csv("datasets/ISO-3166-Countries-with-Regional-Codes.csv", na_filter=None)

In [27]:
df_iso = df_iso[["name", "alpha-2", "alpha-3"]]

In [28]:
df_iso.columns = ["name", "iso_2", "iso_3"]

In [29]:
df_iso

,name,iso_2,iso_3
0,Afghanistan,AF,AFG
1,Åland Islands,AX,ALA
2,Albania,AL,ALB
3,Algeria,DZ,DZA
4,American Samoa,AS,ASM
...,...,...,...
244,Wallis and Futuna,WF,WLF
245,Western Sahara,EH,ESH
246,Yemen,YE,YEM
247,Zambia,ZM,ZMB


In [30]:
df_iso.dtypes

name     object
iso_2    object
iso_3    object
dtype: object

In [31]:
df_iso

,name,iso_2,iso_3
0,Afghanistan,AF,AFG
1,Åland Islands,AX,ALA
2,Albania,AL,ALB
3,Algeria,DZ,DZA
4,American Samoa,AS,ASM
...,...,...,...
244,Wallis and Futuna,WF,WLF
245,Western Sahara,EH,ESH
246,Yemen,YE,YEM
247,Zambia,ZM,ZMB


In [32]:
client.insert_df("datasets.iso_country", df_iso)

In [47]:
print("Type of df:", type(df))
print("Type of df.columns:", type(df.columns))
print("df.columns:", df.columns)
print("Length of df.columns:", len(df.columns))
print("DataFrame shape:", df.shape)
print("DataFrame info:")
df.info()

Type of df: <class 'pandas.core.frame.DataFrame'>
Type of df.columns: <class 'pandas.core.indexes.base.Index'>
df.columns: Index(['country', 'year', 'iso_code', 'population', 'gdp',
       'biofuel_cons_change_pct', 'biofuel_cons_change_twh',
       'biofuel_cons_per_capita', 'biofuel_consumption',
       'biofuel_elec_per_capita',
       ...
       'solar_share_elec', 'solar_share_energy', 'wind_cons_change_pct',
       'wind_cons_change_twh', 'wind_consumption', 'wind_elec_per_capita',
       'wind_electricity', 'wind_energy_per_capita', 'wind_share_elec',
       'wind_share_energy'],
      dtype='object', length=129)
Length of df.columns: 129
DataFrame shape: (11946, 129)
DataFrame info:
<class 'pandas.core.frame.DataFrame'>
Index: 11946 entries, 0 to 22011
Columns: 129 entries, country to wind_share_energy
dtypes: datetime64[ns](1), float64(126), object(2)
memory usage: 11.8+ MB
